<a href="https://colab.research.google.com/github/SurbhiS19/NER_resume_extraction/blob/main/Resume_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_trf

2023-03-31 23:36:42.278156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 23:36:44.294809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-03-31 23:36:48.247385: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-31 23:36:48.248132: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
!nvidia-smi

Fri Mar 31 23:37:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W /  70W |      0MiB / 15360MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from tqdm.autonotebook import tqdm

import spacy
from spacy import displacy

import json

<ipython-input-3-72d24a55f03f>:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
# error while doing git clone because of overriding my locale hence work around is to create a function that returns the required local 
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
!git clone https://github.com/SurbhiS19/NER_resume_extraction.git

fatal: destination path 'NER_resume_extraction' already exists and is not an empty directory.


In [6]:
nlp = spacy.load("en_core_web_trf")

In [7]:
res_data = []
for line in open('/content/NER_resume_extraction/Resume.json','r+',encoding="utf-8"):
  res_data.append(json.loads(line))


In [8]:
res_data[1]['annotation'][5]['points']

[{'start': 612, 'end': 636, 'text': 'B.Sc. in Computer Science'}]

In [9]:
list(res_data[1].keys())

['content', 'annotation']

In [10]:
res=res_data[11]

In [52]:
doc = nlp(res['content'])

In [12]:
len(doc)

844

In [13]:
dir(doc)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_context',
 '_get_array_attrs',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'from_json',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set

In [14]:
displacy.render(doc, style="ent", jupyter=True)

In [53]:
def convert_to_spacy(Json_filepath):
  res_data = []
  for line in open(Json_filepath,'r+',encoding="utf-8"):
    res_data.append(json.loads(line))

  training_data=[]
  for data in res_data:
    text = data['content']
    entities=[]
    for annotation in data['annotation']:
      point = annotation['points'][0]
      labels = annotation['label']
      # handle both list of labels or a single label.
      if not isinstance(labels, list):
        labels = [labels]

      for label in labels:
        entities.append((point['start'], point['end'] + 1 ,label))

    training_data.append((text, {"entities" : entities}))

  return training_data

In [57]:
train_data = convert_to_spacy("/content/NER_resume_extraction/Resume.json")
# train_data[1]

In [58]:
nlp = spacy.blank('en')  # create blank Language class

In [59]:
import random

In [66]:
def train_model(train_data):
  if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
      # nlp.add_pipe(ner, last = True)
      # ner = nlp.add_pipe("ner")
  else:
    ner = nlp.get_pipe("ner")
  
  for _, annotation in train_data:
    for ent in annotation['entities']:
      ner.add_label(ent[2])
          
  
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes): # only train NER
    optimizer = nlp.begin_training()
    for itn in range(30):
      print("Statring iteration " + str(itn))
      random.shuffle(train_data)
      losses = {}
      index = 0
      for text, annotations in train_data:
        try:
          nlp.update(
              [text],  # batch of texts
              [annotations],  # batch of annotations
              drop=0.2,  # dropout - make it harder to memorise data
              sgd=optimizer,  # callable to update weights
              losses=losses)
        except Exception as e:
          pass
          
      print(losses)
  

In [67]:
train_model(train_data)

Statring iteration 0
{}
Statring iteration 1
{}
Statring iteration 2
{}
Statring iteration 3
{}
Statring iteration 4
{}
Statring iteration 5
{}
Statring iteration 6
{}
Statring iteration 7
{}
Statring iteration 8
{}
Statring iteration 9
{}
Statring iteration 10
{}
Statring iteration 11
{}
Statring iteration 12
{}
Statring iteration 13
{}
Statring iteration 14
{}
Statring iteration 15
{}
Statring iteration 16
{}
Statring iteration 17
{}
Statring iteration 18
{}
Statring iteration 19
{}
Statring iteration 20
{}
Statring iteration 21
{}
Statring iteration 22
{}
Statring iteration 23
{}
Statring iteration 24
{}
Statring iteration 25
{}
Statring iteration 26
{}
Statring iteration 27
{}
Statring iteration 28
{}
Statring iteration 29
{}
